# Øving 3 - Numerisk integrasjon

Øvingen har innleveringsfrist fredag 1. februar, kl 11:00. Øvingen blir veiledet i øvingstimen 25. januar.
For å øvingen godkjent må du 1) laste opp en Jupyter Notebook (gjerne basert på denne du nå leser) og
svare på prøve med kontrollspørsmål, begge deler i Blackboard.

**Oppgave 1** Bestem konstanter $R_1, R_2, R_3$ slik at kvadraturformelen

$$
    Q[f] = R_1 f(0) + R_2 f(0.5) + R_3 f(1)
$$

approksimerer 

$$
    \int_0^1 f(x)\,dx
$$

med presisjonsgrad minst lik 2.

**Kontrollspørsmål 1:** Oppgi $ R_2 - \frac12 R_1 +  R_0$
**Svar** Antatt simpsons metode gir dette $R_1=1/6, R_2=2/3 $ og $ R_3=1/6$, hvilket gir $ R_3 - \frac12 R_2 +  R_1 = 0$

**Oppgave 2** (Programmering). 

**a)** Implementer en adaptiv trapesmetode i Python som beskrevet i kapittel 5.4 
(du kan gjerne bare oversette Program 5.2 til Python) i boka. Lag Python funksjonen slik at det teller opp og returnerer antall kall til funksjonen som definerer integranden (typisk kalt $f$).

**b)** Kjør programmet med toleranse tol=1.0e-08 på integralet

$$
   I =  \int_0^1 \frac{e^{x}}{\cos x}\; dx
$$

**Kontrollspørsmål 2:** Hvor mange funksjonskall bruker programmet med den gitte toleransen på dette problemet.

**c)** Test programmet med toleranse 1.0e-07 på problemet

$$
     \int_0^{2\sqrt{2\pi}} \sin x^2\; dx
$$

Sannsynligvis returnerer funksjonen din et galt svar etter 1 iterasjon. Forklar hvorfor, og foreslå en måte å komme rundt problemet på. 

**Svar:** Dette er fordi både $a_0$ og $b_0$ satt inn i $x^2$ blir multiplikasjoner av $\pi$, og $\sin n\pi = 0$, dette tilsier at første approksimasjon skal være lik $0$, men det er den ikke fordi en datamaskin bruker tilnærmede verdier av blandt annet $\pi$ og $\sin$. En måte å gjøre dette på er å flytte $b_0$ slik at dette ikke  skjer slik som gjort under fordi $5$ er tilnærmet lik $2\sqrt{2\pi}$. 

**Svar ekstra:**En annen måte som ser ut til å virke for å komme rundt dette på er teste om første approksimasjon alt er under toleransen for å så anta at dette er feil og sette den manuelt lik toleransen (med toleransen menes her $3*tol$). Kommentar: dette fungere her, men jeg vet ikke om dette fungerer generelt.

**Kontrollspørsmål 3:** Angi antall funksjonskall når du endrer øvre grense i integralet fra $b=2\sqrt{2\pi}$ til
$b=5$.


In [76]:
"""Skriv ditt Python-program og test ut i denne cellen"""
import numpy as np


def f1(x):
    return np.e ** x / np.cos(x)


def f2(x):
    return np.sin(x ** 2)

def trap(f, a, b):
    return (f(a) + f(b)) * (b - a) / 2

def vec_3D_add3(veca, vecb, vecc): #skrevet slik at adapquad kan returner flere verdier lett.
    return [veca[0] + vecb[0] + vecc[0], veca[1] + vecb[1] + vecc[1], veca[2] + vecb[2] + vecc[2]]


def adapquad(f, a, b, tol, oldapp):
    c = (a + b) / 2
    app1 = trap(f, a, c)
    app2 = trap(f, c, b)
    itera = [4, 0, 0] #(vet at trap kaller f 2 ganger)*2
    if np.abs(oldapp - (app1 + app2)) < 3 * tol:
        itera[1:2] = [app1 + app2, 1] #en itterasjon er kjørt
        return itera
    else:
        tol = tol / 2 #oppdaterer toleransen
        return vec_3D_add3(itera, adapquad(f, a, c, tol, app1), adapquad(f, c, b, tol, app2))


def runadapquad(f, a, b, tol, test):
    app = trap(f, a, b)
    if app < 3*tol and test: #Lagt til for å kunne teste løsning for fiks av feil.
        print("Første approksimasjon = ", app)        #antar feil hvis dette stemmer og setter derfor første app på nytt
        app = 3*tol       #Merk! vet ikke om dette fungere for alle funksjoner, men dette ser ut til å være det minste tallet som fungerer for f2
    itera = vec_3D_add3([2, 0, 0], adapquad(f, a, b, tol, app), [0, 0, 0]) #vet at trap over kaller f to ganger
    return itera #Vet at oppgaven kun spør om funksjonskall, med tilnærmet verdi av itergralet og antall iterasjoner er like interessant




   
      



In [77]:
itera1= runadapquad(f1, 0, 1, 1.0e-8, False)
print("2b) Antall kall til f = ", itera1[0], " , Tilnærmet intergarl = ", itera1[1], " , Antall iterasjoner = ", itera1[2])

2b) Antall kall til f =  53270  , Tilnærmet intergarl =  2.2177146898063844  , Antall iterasjoner =  6659


In [78]:
itera2= runadapquad(f2, 0, 2*np.sqrt(2*np.pi), 1.0e-7, False)
print("2c) Antall kall til f = ", itera2[0], " , Tilnærmet intergarl = ", itera2[1], " , Antall iterasjoner = ", itera2[2])

2c) Antall kall til f =  6  , Tilnærmet intergarl =  -8.520839995119704e-15  , Antall iterasjoner =  1


In [79]:
itera3= runadapquad(f2, 0, 2*np.sqrt(2*np.pi), 1.0e-7, True)
print("2c) Antall kall til f = ", itera3[0], " , Tilnærmet intergarl = ", itera3[1], " , Antall iterasjoner = ", itera3[2])

Første approksimasjon =  -1.1361119993492939e-14
2c) Antall kall til f =  234542  , Tilnærmet intergarl =  0.527038343367052  , Antall iterasjoner =  29318


In [80]:
itera4= runadapquad(f2, 0, 5, 1.0e-7, False)
print("2c) Antall kall til f = ", itera4[0], " , Tilnærmet intergarl = ", itera4[1], " , Antall iterasjoner = ", itera4[2])

2c) Antall kall til f =  233862  , Tilnærmet intergarl =  0.527917284798749  , Antall iterasjoner =  29233
